#### Data Merging

In [1]:
import pandas as pd
import numpy as np

In [5]:
#import the datasets.
#data1: 2015 - 2020, data2: 2011 - 2015 
data1 = pd.read_csv('data1.csv')
data2 = pd.read_csv('data2.csv')
climate = pd.read_csv('climate.csv')

In [6]:
data2.shape,data1.shape

((43801, 15), (38005, 15))

In [7]:
data12 = pd.concat([data2,data1])
data12.drop(['wind_direction','wind_spd','temp'],axis=1,inplace=True)

In [8]:
climate.shape,data12.shape

((81816, 28), (81806, 12))

In [9]:
climate.head()

,latitude,longitude,date_utc,doy_utc,hour_utc,time_valid_lcl,offset_min,temp,wetBulb,dewPt,...,windSpd10m,windDir10m,windSpd80m,windDir80m,windSpd100m,windDir100m,precip,snowfall,cldCvr,radSolar
0,51.521,-0.2135,2011-01-01 00:00:00,1,0 days 00:00:00.000000000,1/1/2011 0:00,0,4.111111,3.833333,3.500000,...,5.3,286.0,7.4,289.1,7.7,289.8,0.0,0.0,99,0.0
1,51.521,-0.2135,2011-01-01 01:00:00,1,0 days 01:00:00.000000000,1/1/2011 1:00,0,4.611111,4.277778,3.833333,...,6.0,292.9,9.1,296.0,9.5,296.6,0.0,0.0,99,0.0
2,51.521,-0.2135,2011-01-01 02:00:00,1,0 days 02:00:00.000000000,1/1/2011 2:00,0,4.500000,4.055556,3.611111,...,7.0,285.4,10.4,289.0,10.9,290.0,0.0,0.0,98,0.0
3,51.521,-0.2135,2011-01-01 03:00:00,1,0 days 03:00:00.000000000,1/1/2011 3:00,0,4.333333,3.888889,3.333333,...,8.2,283.8,12.0,287.2,12.5,287.8,0.0,0.0,97,0.0
4,51.521,-0.2135,2011-01-01 04:00:00,1,0 days 04:00:00.000000000,1/1/2011 4:00,0,4.277778,3.777778,3.166667,...,9.0,286.4,13.1,290.1,13.7,290.7,0.0,0.0,97,0.0


In [10]:
climate = climate.iloc[1:81807,:]

In [11]:
climate.shape,data12.shape

((81806, 28), (81806, 12))

In [12]:
climate.rename({'date_utc':'date_time'},axis=1,inplace=True)

In [13]:
#-----------------#
climate.date_time = pd.DatetimeIndex(climate.date_time)
data12.date_time = pd.DatetimeIndex(data12.date_time)

In [14]:
data12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81806 entries, 0 to 38004
Data columns (total 12 columns):
date            81806 non-null object
time            81806 non-null object
nitric_oxide    81806 non-null float64
PM10            81806 non-null float64
PM2.5           81806 non-null float64
date_time       81806 non-null datetime64[ns]
average_PM      81806 non-null float64
year            81806 non-null int64
month           81806 non-null int64
day             81806 non-null int64
season          81806 non-null object
day_of_week     81806 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 8.1+ MB


In [15]:
#pd.concat([climate,data12],axis=1,).index
data = pd.concat([climate.set_index('date_time'),data12.set_index('date_time')], axis=1, join='inner').reset_index()

In [16]:
def conv_dir(deg):
    bound = 45/2
    if deg <=360 and deg>360-bound:
        return 'north'
    elif deg<=360-bound and deg>360-(2*bound):
        return 'north-west'
    elif deg <=270 and deg>270-bound:
        return 'west'
    elif deg <=270-bound and deg>270-(2*bound):
        return 'south-west'
    elif deg  <=180 and deg>180-bound:
        return 'south'
    elif deg<=180-bound and deg>180-(2*bound):
        return 'south-east'
    elif deg <=90 and deg>90-bound:
            return 'east'
    elif deg <=90-bound and deg>90-(2*bound):
            return 'north-east'
    else:
        return 'none'

data['wind_dir'] = data.windDir10m.map(conv_dir)    

In [17]:
data.columns

Index(['date_time', 'latitude', 'longitude', 'doy_utc', 'hour_utc',
       'time_valid_lcl', 'offset_min', 'temp', 'wetBulb', 'dewPt', 'feelsLike',
       'windchill', 'heatind', 'relHum', 'specHum', 'Pres', 'Pres_tndncy',
       'mslPres', 'windSpd10m', 'windDir10m', 'windSpd80m', 'windDir80m',
       'windSpd100m', 'windDir100m', 'precip', 'snowfall', 'cldCvr',
       'radSolar', 'date', 'time', 'nitric_oxide', 'PM10', 'PM2.5',
       'average_PM', 'year', 'month', 'day', 'season', 'day_of_week',
       'wind_dir'],
      dtype='object')

In [19]:
data.to_csv('data.csv',index=False)#final dataframe.